## Load Libraries

In [1]:
import csv, nltk, re, pprint, datetime, random, os, sys
from dateutil import parser
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from nltk.corpus import sentiwordnet as swn
import math
import numpy as np
print ("Complete")

Complete


## Folders & Files Name Defintion

In [2]:
#Folder Names
#STATIC_DATA_FOLDER = "C:/Users/Fred/My Documents/LiClipse Workspace/MarketSentimentAnalysis/data/processor_static_data/"
#TWEETS_FOLDER = "C:/Users/Fred/My Documents/LiClipse Workspace/MarketSentimentAnalysis/data/weekTweets/"
#MKT_PRICE_FOLDER = "C:/Users/Fred/My Documents/LiClipse Workspace/MarketSentimentAnalysis/data/prices/"

STATIC_DATA_FOLDER = "C:/Users/Fred/Desktop/Studies/MSc-DataScience/NLP/Assignment3/deliverables/data/processor_static_data/"
TWEETS_FOLDER = "C:/Users/Fred/Desktop/Studies/MSc-DataScience/NLP/Assignment3/deliverables/data/weekTweets/"
MKT_PRICE_FOLDER = "C:/Users/Fred/Desktop/Studies/MSc-DataScience/NLP/Assignment3/deliverables/data/prices/"

#Static Data File Names
STOP_WORDS = 'stop_words.csv'
CUSTOM_TRADING_CORPUS = 'custom_trading_corpus.csv'
#Mkt File Prices
MERGED_MKT_PRICES = 'MERGED_GOOGL_MKT_PRICE.csv'
#RAW_TWEET_TEST = 'sample-test.csv' 
MERGED_TWEETS = 'MERGED_GOOGL_TWEETS.csv'
PRICE_VARIATION_VS_SENTIMENT = 'PRICE_VARIATION_VS_SENTIMENT.csv'

print("")
print ("Complete")


Complete


## Shared Utilities Functions & Variables

In [3]:
DATE_TIME_FORMAT = '%Y-%m-%d %H:%M:%S'
POSITIVE_SENTIMENT = 'positive'
NEGATIVE_SENTIMENT = 'negative'
NEUTRAL_SENTIMENT = 'neutral'
NA = 'NOT_AVAILABLE'
TRADING_START_HOUR_GMT = "14" 
TRADING_START_MIN_GMT = "30"
TRADING_END_HOUR_GMT = "21" 
TRADING_END_MIN_GMT = "00"
START_TRADING_TIME = int(TRADING_START_HOUR_GMT + TRADING_START_MIN_GMT)
END_TRADING_TIME = int(TRADING_END_HOUR_GMT + TRADING_END_MIN_GMT)
print ("Start Trading Time (GMT): " + str(START_TRADING_TIME)) 
print ("End Trading Time (GMT): " + str(END_TRADING_TIME)) 

def delete_file(file_name_full_path):
    if (os.path.exists(file_name_full_path)):
        os.remove(file_name_full_path)
        
#Get file list from a folder in order of creation
def get_files_in_folder(search_dir, file_name_pattern_match):
    os.chdir(search_dir)
    files = filter(os.path.isfile, os.listdir(search_dir))
    files = [os.path.join("", f) for f in files]
    files = [file_name for file_name in files if file_name.find(file_name_pattern_match) != -1]
    files.sort(key=lambda x: os.path.getmtime(x))
    return files

#Indicates whether the string date of expected format e.g. '2017-01-17 18:39:50' 
#is between the trading hours (i.e. 14.30am and 21.00 GMT
def is_within_trading_hours(date_as_string):
    time_splits = date_as_string.split(" ")[1].split(":")
    hour_minute = int(time_splits[0] + time_splits[1])
    if  (hour_minute >= START_TRADING_TIME and hour_minute <= END_TRADING_TIME):
        return True
    else:
        return False
    
print("")
print ("Complete")

Start Trading Time (GMT): 1430
End Trading Time (GMT): 2100

Complete


## Market Price Pre-processor

### Merge Market Prices Files
Load each market data price file and merge them into a unique file
Note: 
* New York Stock Exchange (NYSE) trading hours: 9.30-16.00EST (EST +5 hours: 14:30-21:00GMT)
* The date timezone is GMT in MERGED_MKT_PRICES csv file

In [4]:
try:
    #Market price source files
    market_price_list = []
    market_price_rows = []
    
    #Delete existing mergedMarketPrice file
    delete_file(MKT_PRICE_FOLDER+MERGED_MKT_PRICES)
    
    #Add all market price files to the list
    files = get_files_in_folder(MKT_PRICE_FOLDER, "GOOGL_2017")
    print('Market Price files to be merged:')
    for file_name in files:
        print(file_name)
        market_price_list.append(file_name)

    #Market price destination file
    f_dest = open(MKT_PRICE_FOLDER+MERGED_MKT_PRICES, 'w', newline='')
    merged_market_price_csv = csv.writer(f_dest, delimiter=',')
    merged_market_price_csv.writerow(['Date','Open','High','Low','Close','Volume'])

    #Stich up all market price files
    last_date = 0
    for file_index, file_name in  enumerate(market_price_list):
        full_file_path = MKT_PRICE_FOLDER + file_name
        f_source = open(full_file_path, 'rt')
        raw_market_price_csv = csv.reader(f_source, delimiter=',')
        for row_index, row in enumerate(raw_market_price_csv):
            #convert dates as int types into string readible dates
            row[0] = datetime.datetime.fromtimestamp(int(row[0])).strftime(DATE_TIME_FORMAT)
            if (file_index == 0):
                #Load the first raw market file into the list
                if (is_within_trading_hours(row[0])):
                    market_price_rows.append(row)
            else:
                #For any other of raw market price file, only store the rows for which
                #the date is > than the last date in the first market price file (to ensure date uniqueness)
                if (row[0] > last_date):
                    if (is_within_trading_hours(row[0])):
                        market_price_rows.append(row)
        #Get last date in the list        
        last_date = market_price_rows[-1][0]
        
    #Save all rows at once
    merged_market_price_csv.writerows(market_price_rows)
except:
    print("Unexpected error:", sys.exc_info()[0])
    pass
finally:
    f_dest.flush()
    f_dest.close()
    f_source.flush()
    f_source.close()
    
print("")
print ("Complete")

Market Price files to be merged:
GOOGL_2017_01_04_.csv
GOOGL_2017_01_05_.csv
GOOGL_2017_01_06_.csv
GOOGL_2017_01_09_.csv
GOOGL_2017_01_10_.csv
GOOGL_2017_01_11_.csv
GOOGL_2017_01_12_.csv
GOOGL_2017_01_13_.csv
GOOGL_2017_01_16_.csv
GOOGL_2017_01_17_.csv
GOOGL_2017_01_18_.csv
GOOGL_2017_01_19_.csv
GOOGL_2017_01_20_.csv
GOOGL_2017_01_21_.csv
GOOGL_2017_01_23_.csv
GOOGL_2017_01_24_.csv
GOOGL_2017_01_25_.csv
GOOGL_2017_01_27_.csv
GOOGL_2017_01_30_.csv
GOOGL_2017_01_31_.csv

Complete


## Sentiment Pre-processor

### Utilities Functions

In [5]:
#initialize stopWords
stop_words = []

#Tweet pre-processing
def process_tweet(tweet):
    #Convert to lower case
    tweet = tweet.lower()
    #remove the characters b'
    tweet = tweet[1:]
    #Remove the 'rt' characters 
    tweet = tweet.strip('rt')
    #Convert www.* or https?://* to URL
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    #Convert @username to AT_USER
    tweet = re.sub('@[^\s]+','AT_USER',tweet)
    #Remove additional white spaces
    tweet = re.sub('[\s]+', ' ', tweet)
    #Replace #word with word
    tweet = re.sub(r'#([^\s]+)', r'\1', tweet)
    #Trim
    tweet = tweet.strip('\'"')
    #Remove single quotes
    tweet = tweet.replace("'", "")
    #Remove double quotes
    tweet = tweet.replace('"', "")
    return tweet

def replace_two_or_more(s):
    #look for 2 or more repetitions of character and replace with the character itself
    pattern = re.compile(r"(.)\1{1,}", re.DOTALL)
    return pattern.sub(r"\1\1", s)

def get_stop_word_list(stop_word_list_fileName):
    #read the stopwords file and build a list
    stop_words = []
    stop_words.append('AT_USER')
    stop_words.append('URL')

    fp = open(stop_word_list_fileName, 'r')
    line = fp.readline()
    while line:
        word = line.strip()
        stop_words.append(word)
        line = fp.readline()
    fp.close()
    return stop_words

def get_feature_vector(tweet, stop_words):
    feature_vector = []
    #split tweet into words
    words = tweet.split()
    for w in words:
        #replace two or more with two occurrences
        w = replace_two_or_more(w)
        #strip punctuation
        w = w.strip('\'"?,.')
        #check if the word stats with an alphabet
        val = re.search(r"^[a-zA-Z][a-zA-Z0-9]*$", w)
        #ignore if it is a stop word
        if(w in stop_words or val is None):
            continue
        else:
            feature_vector.append(w.lower())
    return feature_vector

def convert_date_to_string(row, index, tag):
    dt = parser.parse(remove_uncessary_tag(row,index,tag)).strftime(DATE_TIME_FORMAT)
    return dt

def remove_uncessary_tag(row, index, tag):
    the_str = "NA"
    splits = row[index].split(tag)
    if (len(splits) > 1):
        the_str = row[index].split(tag)[1]
    return the_str

#Check wheter the row first item contains a valid date
def validate(row, index, tag):
    try:
        parser.parse(remove_uncessary_tag(row,index, tag))
        return True
    except ValueError:
        return False 
print("")
print ("Complete")


Complete


### Merge Twitter Files 
Load each tweeer file and merge it into a unique file. 
Notes:
- Ensure only tweet rows starting with a valid date are merged 
- The 'created_at' column is a UTC (i.e. GMT) date as mentioned in https://twittercommunity.com/t/problems-with-created-at-time-zone/9238

In [6]:
try:
    #Tweet source files
    tweet_list = []
    tweet_rows = []
    
    #Delete existing mergedMarketPrice file
    delete_file(TWEETS_FOLDER+MERGED_TWEETS)
    
    #Add all market price files to the list
    files = get_files_in_folder(TWEETS_FOLDER, "GOOGL_today")
    print('Tweet files to be merged:')
    for file_name in files:
        print(file_name)
        tweet_list.append(file_name)

    #Tweets destination file
    f_dest = open(TWEETS_FOLDER+MERGED_TWEETS, 'w', newline='')
    merged_tweets_csv = csv.writer(f_dest, delimiter=',')
    merged_tweets_csv.writerow(['created_at','text'])

    #Stitch up all tweet files
    last_date = "1970-01-01 0:0:0"
    for file_index, file_name in  enumerate(tweet_list):
        full_file_path = TWEETS_FOLDER + file_name
        f_source = open(full_file_path, 'rt')
        raw_tweet_csv = csv.reader(f_source, delimiter=',')
        for row_index, row in reversed(list(enumerate(raw_tweet_csv))):
            #ignore empty row
            if len(row) > 0:
                #normalise the data for all rows <> header and starting with a valid date
                if (row_index > 0 and validate(row,0,"b'")):
                    row[0] = convert_date_to_string(row,0, "b'")
                    #merge data    
                    if (file_index == 0):
                        #Load the entire tweeter file into the merged file
                        if (is_within_trading_hours(row[0])):
                            tweet_rows.append(row)
                    else:
                        #For any other of raw tweet file, only store the rows for which
                        #the date is > than the last date in the first tweet file (to ensure date uniqueness)
                        curr_date_as_date = datetime.datetime.strptime(row[0], DATE_TIME_FORMAT)
                        last_date_as_date = datetime.datetime.strptime(last_date, DATE_TIME_FORMAT)
                        if (row[0] > last_date):
                            if (is_within_trading_hours(row[0])):
                                tweet_rows.append(row)
                    #Get last date in the list
                    if (len(tweet_rows)>0):
                        last_date = tweet_rows[-1][0]
    #Save all rows at once
    merged_tweets_csv.writerows(tweet_rows)
except:
    print("Unexpected error:", sys.exc_info()[0])
    pass
finally:
    f_dest.flush()
    f_dest.close()
    f_source.flush()
    f_source.close()

print("")
print ("Complete")

Tweet files to be merged:
tweet_GOOGL_today_tweets_2017-01-04-06.30.51_today.csv
tweet_GOOGL_today_tweets_2017-01-04-07.25.57_today.csv
tweet_GOOGL_today_tweets_2017-01-04-08.25.17_today.csv
tweet_GOOGL_today_tweets_2017-01-04-09.15.16_today.csv
tweet_GOOGL_today_tweets_2017-01-04-09.25.18_today.csv
tweet_GOOGL_today_tweets_2017-01-04-10.15.16_today.csv
tweet_GOOGL_today_tweets_2017-01-04-10.25.19_today.csv
tweet_GOOGL_today_tweets_2017-01-04-11.15.17_today.csv
tweet_GOOGL_today_tweets_2017-01-04-12.09.52_today.csv
tweet_GOOGL_today_tweets_2017-01-04-12.59.50_today.csv
tweet_GOOGL_today_tweets_2017-01-04-13.59.51_today.csv
tweet_GOOGL_today_tweets_2017-01-04-14.09.53_today.csv
tweet_GOOGL_today_tweets_2017-01-04-14.59.51_today.csv
tweet_GOOGL_today_tweets_2017-01-04-15.09.54_today.csv
tweet_GOOGL_today_tweets_2017-01-04-15.59.52_today.csv
tweet_GOOGL_today_tweets_2017-01-04-16.09.55_today.csv
tweet_GOOGL_today_tweets_2017-01-04-16.59.52_today.csv
tweet_GOOGL_today_tweets_2017-01-04-17.

### Generate Feature Vectors and Corresponding Sentiment

In [7]:
#Get the list of stop words for the English language. 
#The list has  been leveraged from http://xpo6.com/list-of-english-stop-words/
stop_words = get_stop_word_list(STATIC_DATA_FOLDER+ STOP_WORDS)

#Read the Tweets one by one and process it
raw_tweets = csv.reader(open(TWEETS_FOLDER+MERGED_TWEETS, 'rt'), delimiter=',')
tweets = []
for index, row in enumerate(raw_tweets):
    if (index > 0):
        #Do a sanity check
        if ( len(row) != 2 ):
            raise Exception('the length must be 2, it cannot be: ' + str(len(row)) + 'at row: ' + str(row))
        #Start the vectorisation process
        sentiment = row[0]      
        tweet = row[1]
        #1) Normalise the Tweets
        processed_tweet = process_tweet(tweet)
        #2) Generate the feature vector
        feature_vector = get_feature_vector(processed_tweet, stop_words)
        #3) Generate the dataset
        tweets.append((feature_vector, sentiment))
    
print("Tweets bag of words: " )
print("")
print (tweets)
print("")
print ("Complete")

Tweets bag of words: 

[(['fascinating', 'life', 'google', 'founder', 'alphabet', 'ceo', 'larry', 'page', 'business'], '2017-01-04 14:30:07'), (['fascinating', 'life', 'google', 'founder', 'alphabet', 'ceo', 'larry', 'page'], '2017-01-04 14:30:20'), (['fascinating', 'life', 'google', 'founder', 'alphabet', 'ceo', 'larry', 'page'], '2017-01-04 14:30:54'), (['fascinating', 'life', 'google', 'founder', 'alphabet', 'ceo', 'larry', 'page'], '2017-01-04 14:31:23'), (['fascinating', 'life', 'google', 'founder', 'alphabet', 'ceo', 'larry', 'page'], '2017-01-04 14:32:13'), (['fascinating', 'life', 'google', 'founder', 'alphabet', 'ceo', 'larry', 'page'], '2017-01-04 14:32:24'), (['fascinating', 'life', 'google', 'founder', 'alphabet', 'ceo', 'larry', 'page', 'business'], '2017-01-04 14:33:27'), (['fascinating', 'life', 'google', 'founder', 'alphabet', 'ceo', 'larry', 'page'], '2017-01-04 14:33:36'), (['setting4success', 'fascinating', 'life', 'google', 'founder', 'alphabet', 'ceo', 'larry', 'pa

### Sentiment Generation

see http://stackoverflow.com/questions/38263039/sentiwordnet-scoring-with-python/38263475 for more info
http://www.nltk.org/_modules/nltk/corpus/reader/sentiwordnet.html

In [8]:
#Read the Tweets one by one and process it
custom_trading_corpus = csv.reader(open(STATIC_DATA_FOLDER+CUSTOM_TRADING_CORPUS, 'rt'), delimiter=',')
custom_trading_corpus_deep_copy = list(custom_trading_corpus)    
f_dest = open(TWEETS_FOLDER+"DEBUG_SENTIMENT", 'w', newline='')
debug_sentiment_csv = csv.writer(f_dest, delimiter=',')

#The set including the sentiment for each tweet 
tweets_with_sentiments =[]
#Get sentiment from nltk sentiwordnet
for index, row in enumerate(tweets):
    bag_of_words = row[0]
    count_neutral = 0
    count_positive = 0
    count_negative = 0
    for word in bag_of_words:
        bFound = False
        sentiment = NA
        word = word.strip()
        #Look for the sentiment in the custom trading corpus
        #and increment the one of the counters (positive,negative,neutral)
        for row_ctc in custom_trading_corpus_deep_copy:
            if row_ctc[0].strip() == word:
                if row_ctc[2].strip() == POSITIVE_SENTIMENT:
                    count_positive += 1
                    bFound = True
                elif row_ctc[2].strip() == NEGATIVE_SENTIMENT:
                    count_negative += 1
                    bFound = True
                elif row_ctc[2].strip() == NEUTRAL_SENTIMENT:
                    count_neutral += 1
                    bFound = True
                else:
                    bFound = False
        
        if (bFound == True):
            debug_sentiment_csv.writerow ([word + " $ in trading corpus/sentiment: $ " + str(row_ctc[2])])
            #print ([word + " $ in trading corpus/sentiment: $ " + str(row_ctc[2])])
        else:
            #If it is not found in the custom traiding corpus, 
            #then get the sentiment from the sentiwordnet corpus
            senti = swn.senti_synsets(word)
            senti_synsets = list(senti)
            if (len(senti_synsets) > 0):
                # Take the first sense, the most common
                senti_synset = senti_synsets[0]
                #when there is a case where the the word sentiment is proba is split 50/50, 
                #the positive or negative sentiment is chosen 
                if (senti_synset.pos_score() >= 0.5):
                    sentiment = POSITIVE_SENTIMENT
                    count_positive += 1 
                elif (senti_synset.neg_score() >= 0.5): 
                    sentiment = NEGATIVE_SENTIMENT
                    count_negative += 1
                else:
                    #This means that when a sentiwordnet have a positive (negative) =0.5 and neutral = 0.5,
                    #the neutral value is selected (this is a cautious approach)
                    sentiment = NEUTRAL_SENTIMENT
                    count_neutral += 1
                debug_sentiment_csv.writerow ([word + " $ in sentiwordnet corpus/sentiment: $ " + sentiment])
                #print ([word + " $ in sentiwordnet corpus/sentiment: $ " + sentiment])
            else:
                #If word not found, then set to neutral as default
                #score = 'neutral'
                sentiment = NEUTRAL_SENTIMENT
                count_neutral += 1
                debug_sentiment_csv.writerow ([word + " $ not present in any corpa/sentiment: $ " + sentiment])
                #print ([word + " $ not present in any corpa/sentiment: $ " + sentiment])
        #debug_sentiment_csv.writerow([word + " - count " + POSITIVE_SENTIMENT + ":"  + str(count_positive) + 
        #             " - count " + NEGATIVE_SENTIMENT + ":" + str(count_negative) + 
        #             " - count " + NEUTRAL_SENTIMENT + ":"  + str(count_neutral)] )
            
    sentiment_scores = {POSITIVE_SENTIMENT: count_positive, 
                        NEUTRAL_SENTIMENT: count_neutral, 
                        NEGATIVE_SENTIMENT: count_negative}
    debug_sentiment_csv.writerow(["sentiment_scores: " + str(sentiment_scores)])
    
    #Tweeter data analysis showed that there is a large skew towards neutral words in a bag of words.
    #In other words, most sentences contains a higher frequency of neutral words in a sentence than 
    #positive or negative. Therefore, there is a need to re-balance this.

    #Therefore, the neutral category should only be taken into consideration if there is no positive(negative) sentiment
    count_positive = sentiment_scores.get(POSITIVE_SENTIMENT)
    count_neutral = sentiment_scores.get(NEUTRAL_SENTIMENT)
    count_negative = sentiment_scores.get(NEGATIVE_SENTIMENT)
    
    #When there is the same number of positive and negative word sentiments 
    #=> the bag of words sentiment is neutral
    #When there is a greater counter of positive (negative) word sentiments 
    #=> the bag of words sentiment is postive (negative)
    sentiment_bag_of_words = NEUTRAL_SENTIMENT
    if (count_positive > count_negative):
            sentiment_bag_of_words = POSITIVE_SENTIMENT
    elif (count_positive < count_negative):
            sentiment_bag_of_words = NEGATIVE_SENTIMENT

    #Each row contains the bags of word, the sentiment and the datetime stamp
    tweets_with_sentiments.append([row[0],sentiment_bag_of_words, row[1]])
    
    #Find the most frequent sentiment in a bag of words
    #sentiment_bag_of_words = max(sentiment_scores, key=sentiment_scores.get)
    debug_sentiment_csv.writerow (["**********************************"])
    debug_sentiment_csv.writerow (["***Analysis bag of words: " + str(row) + " / sentiment: " + sentiment_bag_of_words])
    debug_sentiment_csv.writerow (["**********************************"])
    #print("***Analysis bag of words: " + str(row) + " / sentiment: " + sentiment_bag_of_words)
    
print("")    
print ("Complete")


Complete


## Perform Analysis

### Analysis Dataset Creation

In [13]:
#Generate the moving average for a period
def movingaverage (values, window):
    weights = np.repeat(1.0, window)/window
    sma = np.convolve(values, weights, 'valid')
    return sma

#Load prices in a dataframe
raw_prices_df = pd.read_csv(MKT_PRICE_FOLDER+MERGED_MKT_PRICES)

#Avg = (High + Low + Close)/3
raw_prices_df['Average']= raw_prices_df.apply(lambda row :
                          (row['High']+row['Low']+row['Close'])/3, 
                          axis=1)
#Generate the Simple Moving Average
sam1 = movingaverage(raw_prices_df['Average'],1)
sam5 = movingaverage(raw_prices_df['Average'],5)
sam20 = movingaverage(raw_prices_df['Average'],20)
#Default new columns
raw_prices_df['%Move'] = 0
raw_prices_df['%Abs_Move'] = 0
raw_prices_df['Up_Down'] = 0
raw_prices_df['Log_Return'] = 0
raw_prices_df['Log_Return_M1'] = 0
raw_prices_df['Sma_1'] = 0
raw_prices_df['Sma_5'] = 0
raw_prices_df['Sma_20'] = 0
raw_prices_df['Up_Down_Value'] = -999
raw_prices_df['Sentiment_Value'] = -999
sma1Idx = 0
sma5Idx = 0
sma20Idx = 0
#Epsilon is use as a mechanism to generate more market neutral moves.
epsilon = 0.0001

tweet_row_index=0
prev_avg_price =0
prev_price_datetime = datetime.datetime.strptime("1970-01-01 00:00:00", DATE_TIME_FORMAT)
for index, price_row in raw_prices_df.iterrows():
    price_datetime = datetime.datetime.strptime(price_row['Date'], DATE_TIME_FORMAT)
    avg_price = price_row['Average']
    count_positive = 0
    count_negative = 0
    count_neutral = 0
    for tweet_row in tweets_with_sentiments[tweet_row_index:]:
        tweet_datetime = datetime.datetime.strptime(tweet_row[2], DATE_TIME_FORMAT)
        if ((tweet_datetime >= prev_price_datetime and tweet_datetime < price_datetime)):
            if tweet_row[1].strip() == POSITIVE_SENTIMENT:
                count_positive += 1
            elif tweet_row[1].strip() == NEGATIVE_SENTIMENT:
                count_negative += 1
            elif tweet_row[1].strip() == NEUTRAL_SENTIMENT:
                count_neutral += 1
            tweet_row_index +=1
        else:
            sentiment_scores = {POSITIVE_SENTIMENT: count_positive, 
                                NEUTRAL_SENTIMENT: count_neutral, 
                                NEGATIVE_SENTIMENT: count_negative}

            count_positive = sentiment_scores.get(POSITIVE_SENTIMENT)
            count_neutral = sentiment_scores.get(NEUTRAL_SENTIMENT)
            count_negative = sentiment_scores.get(NEGATIVE_SENTIMENT)

            time_period_sentiment = NEUTRAL_SENTIMENT
            if (count_positive > count_negative):
                    time_period_sentiment = POSITIVE_SENTIMENT
            elif (count_positive < count_negative):
                    time_period_sentiment = NEGATIVE_SENTIMENT
            
            if (prev_avg_price !=0):
                if (index > 1 ):
                    raw_prices_df.loc[index-1,'Volume_M1'] =  raw_prices_df.loc[index-2,'Volume']                        
                raw_prices_df.loc[index,'Sentiment'] = time_period_sentiment
                if (raw_prices_df.loc[index,'Sentiment'] == POSITIVE_SENTIMENT):
                    raw_prices_df.loc[index,'Sentiment_Value'] = 1
                elif (raw_prices_df.loc[index,'Sentiment'] == NEGATIVE_SENTIMENT):
                    raw_prices_df.loc[index,'Sentiment_Value'] = -1 
                else:
                    raw_prices_df.loc[index,'Sentiment_Value'] = 0
                if (index > 1 ):
                    raw_prices_df.loc[index-1,'Sentiment_Value_M1'] =  raw_prices_df.loc[index-2,'Sentiment_Value']                        
                move = 100 * (avg_price - prev_avg_price) / prev_avg_price
                raw_prices_df.loc[index,'%Move'] = move
                raw_prices_df.loc[index,'%Abs_Move'] = abs(move)
                if (index < len(raw_prices_df)):
                    #equivalent to log (Pt+1/Pt), as we are updating index-1
                    raw_prices_df.loc[index-1,'Log_Return'] =  math.log10(avg_price/raw_prices_df.loc[index-1,'Average'])
                if (index > 1 ):
                    raw_prices_df.loc[index-1,'Log_Return_M1'] =  raw_prices_df.loc[index-2,'Log_Return']
                if (index > 0):
                    raw_prices_df.loc[index,'Sma_1'] = sam1[sma1Idx]
                    sma1Idx = sma1Idx +1
                if (index > 4):
                    raw_prices_df.loc[index,'Sma_5'] = sam5[sma5Idx]
                    sma5Idx = sma5Idx +1
                if (index > 19):
                    raw_prices_df.loc[index,'Sma_20'] = sam20[sma20Idx]
                    sma20Idx = sma20Idx +1
                if (raw_prices_df.loc[index-1,'Log_Return'] > epsilon):
                    raw_prices_df.loc[index-1,'Up_Down'] = 'positive' #'up'
                    raw_prices_df.loc[index-1,'Up_Down_Value'] = 1 #'up'
                elif raw_prices_df.loc[index-1,'Log_Return'] < -epsilon:
                    raw_prices_df.loc[index-1,'Up_Down'] = 'negative' #'down'
                    raw_prices_df.loc[index-1,'Up_Down_Value'] = -1 #'down'
                else:
                    raw_prices_df.loc[index-1,'Up_Down'] = 'neutral' #'flat'
                    raw_prices_df.loc[index-1,'Up_Down_Value'] = 0 #'down'
                    
    
            prev_price_datetime = price_datetime
            prev_avg_price = avg_price
            break

#Now remove missing rows generated by Log_Returns and Sma
raw_prices_df = raw_prices_df.drop(raw_prices_df.index[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,len(raw_prices_df)-1]])

#Save to CSV
raw_prices_df.to_csv(TWEETS_FOLDER+PRICE_VARIATION_VS_SENTIMENT, sep=',')

print ("Complete") 

Complete


### Train/Test Dataset Creation

In [14]:
train_ratio = .7
train_count =int(len(raw_prices_df) * train_ratio)
test_count = len(raw_prices_df)-train_count
print ("TrainCount: " + str(train_count))
print ("TestCount: " + str(test_count))
train = raw_prices_df[:train_count]
test = raw_prices_df[train_count:]


TrainCount: 1745
TestCount: 749


### Naive Bayes Sentiment Prediction

In [28]:
from sklearn import datasets
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

def NaiveBayesPrediction(train_X, train_Y, test_X, test_Y):
    # fit a Naive Bayes model to the data
    model = GaussianNB()
    model.fit(train_X, train_Y)
    print(model)
    # make predictions
    expected = test_Y
    predicted = model.predict(test_X)
    # summarize the fit of the model
    print(metrics.classification_report(expected, predicted))
    print(metrics.confusion_matrix(expected, predicted))

print ("1. Generate the Naive Bayes Training model and Test results " + 
       "without the sentiment information\n")

train_X = train[['Log_Return_M1','Sma_5','Volume_M1']]
train_Y = train[:train_count][['Up_Down_Value']]
test_X = test[['Log_Return_M1','Sma_5','Volume_M1']]
test_Y = test[:train_count][['Up_Down_Value']]

NaiveBayesPrediction(train_X, train_Y, test_X, test_Y)

print ("")
print ("*************************************************")
print ("")

print ("2. Generate the Naive Bayes Training model and Test results " +
       "with the sentiment information\n")

train_X = train[['Log_Return_M1','Sma_5','Volume_M1','Sentiment_Value_M1']]
train_Y = train[:train_count][['Up_Down_Value']]
test_X = test[['Log_Return_M1','Sma_5','Volume_M1','Sentiment_Value_M1']]
test_Y = test[:train_count][['Up_Down_Value']]

NaiveBayesPrediction(train_X, train_Y, test_X, test_Y)

print ("Complete")

1. Generate the Naive Bayes Training model and Test results without the sentiment information

GaussianNB(priors=None)
             precision    recall  f1-score   support

         -1       0.31      0.45      0.37       253
          0       0.43      0.29      0.35       273
          1       0.40      0.34      0.37       223

avg / total       0.38      0.36      0.36       749

[[115  60  78]
 [155  80  38]
 [103  44  76]]

*************************************************

2. Generate the Naive Bayes Training model and Test results with the sentiment information

GaussianNB(priors=None)
             precision    recall  f1-score   support

         -1       0.31      0.45      0.37       253
          0       0.44      0.30      0.36       273
          1       0.40      0.35      0.37       223

avg / total       0.38      0.37      0.37       749

[[115  59  79]
 [154  82  37]
 [101  45  77]]
Complete


C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Support Vector Machine (SVM) Sentiment Prediction

In [25]:
# Support Vector Machine
from sklearn import datasets
from sklearn import metrics
from sklearn.svm import SVC

def SVMPrediction(train_X, train_Y, test_X, test_Y, kernel_type):
    # fit a SVM model to the data
    model = SVC(kernel=kernel_type)
    model.fit(train_X, train_Y)
    print(model)
    # make predictions
    expected = test_Y
    predicted = model.predict(test_X)
    # summarize the fit of the model
    print(metrics.classification_report(expected, predicted))
    print(metrics.confusion_matrix(expected, predicted))

print ("1. Generate the SVM Training model and Test results "+ 
       " without the sentiment information\n")

train_X = train[['Log_Return_M1','Sma_5','Volume_M1']]
train_Y = train[:train_count][['Up_Down_Value']]
test_X = test[['Log_Return_M1','Sma_5','Volume_M1']]
test_Y = test[:train_count][['Up_Down_Value']]

SVMPrediction(train_X, train_Y, test_X, test_Y, 'rbf')

print ("")
print ("*************************************************")
print ("")

print ("2. Generate the SVM Training model and Test results " +
       " with the sentiment information\n")

train_X = train[['Log_Return_M1','Sma_5','Volume_M1','Sentiment_Value_M1']]
train_Y = train[:train_count][['Up_Down_Value']]
test_X = test[['Log_Return_M1','Sma_5','Volume_M1','Sentiment_Value_M1']]
test_Y = test[:train_count][['Up_Down_Value']]

SVMPrediction(train_X, train_Y, test_X, test_Y, 'rbf')


print ("Complete")


1. Generate the SVM Training model and Test results  without the sentiment information



C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
             precision    recall  f1-score   support

         -1       0.10      0.00      0.01       253
          0       0.33      0.01      0.01       273
          1       0.29      0.97      0.45       223

avg / total       0.24      0.29      0.14       749

[[  1   2 250]
 [  4   2 267]
 [  5   2 216]]

*************************************************

2. Generate the SVM Training model and Test results  with the sentiment information



C:\Program Files\Anaconda3\lib\site-packages\sklearn\utils\validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
             precision    recall  f1-score   support

         -1       0.10      0.00      0.01       253
          0       0.33      0.01      0.01       273
          1       0.29      0.97      0.45       223

avg / total       0.24      0.29      0.14       749

[[  1   2 250]
 [  4   2 267]
 [  5   2 216]]
Complete
